In [1]:
# https://stackoverflow.com/questions/45741878/using-binary-crossentropy-loss-in-keras-tensorflow-backend
# https://www.machinecurve.com/index.php/2019/10/22/how-to-use-binary-categorical-crossentropy-with-keras/

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
from keras.callbacks import CSVLogger
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext tensorboard
import datetime


#tf.config.experimental.list_physical_devices('GPU')

# RNN based Models (RNN & LSTM & GRU) using tensorflow keras

In [2]:
!rm -rf ./logs/RNN #dont run this line unless you want to delete the tensorboard logs

In [3]:
def convert_3D(matrix):
    return np.expand_dims(matrix, -1)

In [4]:
with open("../archive/ProcessData.npy", 'rb') as f:
        training_data_x = np.load(f)
        test_x = np.load(f)
        training_data_y = np.load(f)
        test_y = np.load(f)
        
training_data_x = convert_3D(training_data_x) 
test_x = convert_3D(test_x) 

training_data_x, validation_data_x, training_data_y, validation_data_y = train_test_split(training_data_x, training_data_y, test_size=0.3)

In [5]:
RNNmodel = Sequential([
    SimpleRNN(32, input_shape=(training_data_x.shape[1:]), activation='relu', return_sequences=True),
    SimpleRNN(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

RNNmodel.compile(
    loss=BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=0.00001),
    metrics=["accuracy", tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.FalseNegatives(),
             tf.keras.metrics.FalsePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.TruePositives()]
)

log_dir = "logs/RNN/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

csv_logger = CSVLogger('keras-models/KerasRNN/training.csv',separator=",", append=False)

RNNmodel.fit(training_data_x, training_data_y, epochs=3, validation_data=(validation_data_x,validation_data_y), callbacks=[tensorboard_callback, csv_logger])

model = RNNmodel
model.save('keras-models/KerasRNN')

Epoch 1/3
112/112 [==============================] - 492s 4s/step - loss: 1.0288 - accuracy: 0.5233 - auc: 0.5586 - precision: 0.0077 - recall: 0.5000 - false_negatives: 13.0000 - false_positives: 1684.0000 - true_negatives: 1850.0000 - true_positives: 13.0000 - val_loss: 0.9578 - val_accuracy: 0.5809 - val_auc: 0.4708 - val_precision: 0.0063 - val_recall: 0.3636 - val_false_negatives: 7.0000 - val_false_positives: 633.0000 - val_true_negatives: 883.0000 - val_true_positives: 4.0000
Epoch 2/3
112/112 [==============================] - 464s 4s/step - loss: 0.9331 - accuracy: 0.6132 - auc: 0.5749 - precision: 0.0087 - recall: 0.4615 - false_negatives: 14.0000 - false_positives: 1363.0000 - true_negatives: 2171.0000 - true_positives: 12.0000 - val_loss: 0.8710 - val_accuracy: 0.6601 - val_auc: 0.4627 - val_precision: 0.0058 - val_recall: 0.2727 - val_false_negatives: 8.0000 - val_false_positives: 511.0000 - val_true_negatives: 1005.0000 - val_true_positives: 3.0000
Epoch 3/3
112/112 [====

In [6]:
%tensorboard --logdir logs/RNN/fit